In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
0 - подсолнечник
1 - картофель
2 - пшеница озимая
3 - гречиха
4 - кукуруза
5 - пшеница яровая
6 - сахарная свекла

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [ ]:
# загрузка данных
df = pd.read_csv("/content/drive/MyDrive/Fields_Inop/last_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Fields_Inop/last_test.csv")

Патчинг предобработанных данных

In [ ]:
#region patch
reg_patch = {133:"Тульская",
             433:"Казахстан",
             644:"Туркменистан",
             797:"Брянская",
             1121:"Брянская",
             1218:"Татарстан",
             1897:"Татарстан",
             2280:"Узбекистан",
             3455:"Туркменистан",
             3484:"Липетская",
             3858:"Татарстан",
             3969:"Украина",
             4426:"Туркменистан",
             4565:"Туркменистан",
             4779:"Туркменистан",
             }
for k in reg_patch:
    print (k,reg_patch[k])
    df["region"][k]=reg_patch[k]

In [ ]:
#full_region patch
df['full_region'][1351]= "Аргамач-Пальна, Липецкая область, Россия, 399758"
df['full_region'][1417]="Хворостянка, Липецкая область, Россия, 399419"
df['full_region'][2214]="«Дон», Тульская область, Россия"
df['full_region'][3484]="Хворостянский сельсовет, Добренский район, Россия"

Расширение данных

In [ ]:
pip install -q  reverse-geocode

In [ ]:
import reverse_geocode as rg
import json
from tqdm import tqdm
g_d =[]
items= df[".geo"].tolist()
for item in tqdm(items):
    data = json.loads(item)
    if data["type"] == "Polygon":
        #print (data["coordinates"][0][0])
        coords= data["coordinates"][0][0]
        latitude = coords[1]
        longitude = coords[0]
        coordinates=(latitude,longitude)
        location = rg.search([coordinates])
        g_d.append(location[0]["city"])
    elif data["type"] == "MultiPolygon":
        coords= data["coordinates"][0][0][0]
        latitude = coords[1]
        longitude = coords[0]
        coordinates=(latitude,longitude)
        location = rg.search([coordinates])
        g_d.append(location[0]["city"])
    elif data["type"] == "GeometryCollection":
        geom = data["geometries"]
        coords= geom[0]["coordinates"][0]
        latitude = coords[1]
        longitude = coords[0]
        coordinates=(latitude,longitude)
        location = rg.search([coordinates])
        g_d.append(location[0]["city"])
    else:
        g_d.append("ошибка")
        print ("неизвестный тип данных geojson ошибка")

In [ ]:
#Город поставки
df["city"]=g_d

In [ ]:
import json
from tqdm import tqdm
def get_coords (df_geo):
    lat =[]
    lon = []
    items= df_geo[".geo"].tolist()
    for item in items:
        data = json.loads(item)
        if data["type"] == "Polygon":
            #print (data["coordinates"][0][0])
            coords= data["coordinates"][0][0]
            latitude = coords[1]
            longitude = coords[0]
            lat.append(latitude)
            lon.append(longitude)
        elif data["type"] == "MultiPolygon":
            coords= data["coordinates"][0][0][0]
            latitude = coords[1]
            longitude = coords[0]
            lat.append(latitude)
            lon.append(longitude)
            
        elif data["type"] == "GeometryCollection":
            geom = data["geometries"]
            coords= geom[0]["coordinates"][0]
            latitude = coords[1]
            longitude = coords[0]
            lat.append(latitude)
            lon.append(longitude)
            
        else:
            print ("неизвестный тип данных geojson")
    return lat,lon

In [ ]:
#Вычисление медианы координат
import statistics
def get_geomedian(lat,lon):
    median=[]
    get_coords
    for lati,longi in zip (lat,lon):
        median.append (statistics.median([lati,longi]))
    return median

lat,lon= get_coords(df)
df["geo_median"]=get_geomedian(lat,lon)
lat,lon= get_coords(df_test)
df_test["geo_median"]=get_geomedian(lat,lon)

In [ ]:
#Создание Full_region features
from geopy.geocoders import Nominatim
import json
# initialize Nominatim API 
geolocator = Nominatim(user_agent="geoapifields")
from tqdm import tqdm
g_d =[]
items= df[".geo"].tolist()
for item in tqdm(items):
    data = json.loads(item)
    if data["type"] == "Polygon":
        #print (data["coordinates"][0][0])
        coords= data["coordinates"][0][0]
        latitude = str(coords[1])
        longitude = str(coords[0])
        location = geolocator.reverse(latitude+","+longitude)
        g_d.append(location[0])
    elif data["type"] == "MultiPolygon":
        coords= data["coordinates"][0][0]
        latitude = str(coords[1])
        longitude = str(coords[0])
        location = geolocator.reverse(latitude+","+longitude)
        g_d.append(location[0])
    elif data["type"] == "GeometryCollection":
        geom = data["geometries"]
        coords= geom[0]["coordinates"][0]
        latitude = str(coords[1])
        longitude = str(coords[0])
        location = geolocator.reverse(latitude+","+longitude)
        g_d.append(location[0])
    else:
        g_d.append("ошибка")
        print ("неизвестный тип данных geojson")
df_test["full_region"] = g_d
print (df_test['full_region'].isnull().sum())
df_test["full_region"].fillna('ошибка', inplace=True)
df_test.to_csv("/content/drive/MyDrive/Fields_Inop/new_test.csv",index=False)

++++++ CatBoost

In [ ]:
!pip -q install catboost

In [ ]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Датасет для обучения
X = df.drop(["id","full_region",".geo","crop","area_embeddings","text_len"], axis = 1)
y = df['crop']

Сначала нам нужно сгенерировать список индексов столбцов, содержащих категориальные данные.

In [ ]:
def get_categorical_indicies(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        #elif col=="reg_vectors":
           # pass
        else:
            cats.append(col)
    
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
    print (cats)
    return cat_indicies
categorical_indicies = get_categorical_indicies(X)

Второе, что нам нужно сделать, это преобразовать все категориальные столбцы в тип данных категории, который требуется Catboost.

In [ ]:
def convert_cats(X):
    cats = []
    for col in X.columns:
        if is_numeric_dtype(X[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        X[col] = X[col].astype('category')
convert_cats(X)

Нам нужно разделить наши данные на два набора данных для обучения и тестирования.

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.25, 
                                                 random_state=63,
                                                 stratify=y,
                                                 #shuffle=False
                                                 )

Обучение

In [ ]:
train_dataset = cb.Pool(X_train,y_train, 
                        cat_features=categorical_indicies,
                        #text_features=text_features,
                        #embedding_features=embedding_features
                        )                                                      
test_dataset = cb.Pool(X_test,y_test,           
                       cat_features=categorical_indicies,
                       #text_features=text_features,
                       #embedding_features=embedding_features
                       )

In [ ]:
model = cb.CatBoostClassifier(loss_function='MultiClass',  
                              eval_metric='Accuracy',
                              od_type='Iter',
                              od_wait=1500,
                              depth=7,
                              iterations=3000,
                              learning_rate=0.1,
                              l2_leaf_reg=30,
                              #bootstrap_type='No', #'MVS', #'Bayesian', #'Bernoulli',
                              random_seed=63, 
                              bootstrap_type="No",
                              sampling_unit="Group",
                              max_ctr_complexity=4,
                              #subsample=0.5, 
                              #bagging_temperature=0.5,
                              #random_strength=0.5,
                              #one_hot_max_size=7,
                              leaf_estimation_method="Newton", #'Gradient', #''                    
                              )

In [ ]:
model.fit(X=train_dataset,
          eval_set=test_dataset,
          #plot_file="/content/drive/MyDrive/Fields_Inop/model_train.jpg",
          #plot=True,
          use_best_model=True,
          verbose=50,
          #logging_level="Info",
         
          )

Результат

In [ ]:
pred = model.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
Мы можем проанализировать, какое влияние оказали наши функции, построив график важности функций.
import numpy as np
def plot_feature_importance(importance,names,model_type):
   feature_importance = np.array(importance)
   feature_names = np.array(names)
   data={'feature_names':feature_names,
         'feature_importance':feature_importance}
   fi_df = pd.DataFrame(data) 
   fi_df.sort_values(by=['feature_importance'],    
                     ascending=False,inplace=True)
   plt.figure(figsize=(15,15))
   sns.barplot(x=fi_df['feature_importance'], 
               y=fi_df['feature_names'])
   plt.title(model_type + ' FEATURE IMPORTANCE')
   plt.xlabel('FEATURE IMPORTANCE')
   plt.ylabel('FEATURE NAMES')
plot_feature_importance(model.get_feature_importance(),X.columns,'CATBOOST')

In [ ]:
#Сохранение модели
model.save_model("/content/drive/MyDrive/Fields_Inop/last_model_98nan.cbm",
           format="cbm",
           export_parameters=None,
           pool=None)

In [ ]:
#Загрузка модели
model = cb.CatBoostClassifier()
model.load_model("/content/drive/MyDrive/Fields_Inop/last_model_97nan.cbm")